# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [ ]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

```sql
SELECT e.employee_name
FROM employees e
JOIN salary s ON e.employee_id = s.employee_id
ORDER BY s.salary_amount DESC
LIMIT 1;
```


In [ ]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the employee who is best paid by joining the `employees` table with the `salary` table on the `ID_usr` column. It then orders the result by the salary in descending order and limits the output to only one row, which corresponds to the employee with the highest salary.


In [38]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

```sql
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_Usr = e.ID_Usr
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```


In [39]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql
SELECT s.Institution
FROM studies s
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY AVG(sa.salary) DESC
LIMIT 1;
```

This SQL query joins the "studies" and "salary" tables on the ID_usr column. It then calculates the average salary for each institution, orders the results in descending order based on the average salary, and returns the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [9]:
print(return_CCRMSQL("Retrieve the employee with the highest salary and the highest educational level", context_user))


```sql
SELECT e.employee_name, e.salary, e.educational_level
FROM employees e
WHERE e.salary = (SELECT MAX(salary) FROM employees)
AND e.educational_level = (SELECT MAX(educational_level) FROM employees);
```


In [10]:
print(return_CCRMSQL("Retrieve the employee with the highest salary and an average experience level", context_user))

```sql
WITH max_salary_employee AS (
    SELECT *
    FROM employees
    ORDER BY salary DESC
    LIMIT 1
),
avg_experience_level AS (
    SELECT AVG(experience_level) AS avg_experience
    FROM employees
)
SELECT max_salary_employee.*, avg_experience
FROM max_salary_employee, avg_experience_level;
```


In [11]:
print(return_CCRMSQL("Which employee has the highest performance rating?", context_user))

```sql
SELECT employee_name
FROM employee_performance
WHERE performance_rating = (SELECT MAX(performance_rating) FROM employee_performance);
```


## Summary of Findings
I wrote SQL queries to find employees with the highest salary, highest education level, average experience, and highest performance rating. Here’s what I found:
 1. Employee with the highest salary and highest educational level
    - Issue: The query assumes the highest salary and highest education level belong to the same employee. If no such employee exists, it will return no results.
 2. Employee with the highest salary and average experience 
    - Issue: If experience is in a different table, the query will need an extra join. Also, if no employee has the exact average experience, it may return no results.
 3. Employee with the highest performance rating
    - Issue: If multiple employees have the same highest performance rating, all will be returned. To get only one, you would need to add LIMIT 1.






